In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
data = json.load(open("raw_comment_responses.json"))

In [3]:
comment_data = data['comments']

In [19]:
len(comment_data)

6650

In [5]:
videoIds = [list(comment.keys())[0] for comment in comment_data]

In [6]:
# Pass in a video (JSON response from YouTube API) return a dataframe
def getCommentsFromVideo(video):
    video_id = list(video.keys())[0]
    comments = list(video.values())[0]
    if 'noComments' in comments.keys():
        return
        # return pd.DataFrame({
        #     'videoId': video_id,
        #     'textDisplay': "",
        #     'textOriginal': "",
        #     'authorDisplayName': "",
        #     'authorProfileImageUrl': "",
        #     'authorChannelUrl':"",
        #     'authorChannelId':{'value':""},
        #     'canRate':False,
        #     'viewerRating':"",
        #     'likeCount':0,
        #     'publishedAt':"",
        #     'updatedAt':"",
        #     'commentId':""
        # })
    else:
        comments = comments['items']

    if len(comments) == 0:
        return
        # return pd.DataFrame({
        #     'videoId': video_id,
        #     'textDisplay': "",
        #     'textOriginal': "",
        #     'authorDisplayName': "",
        #     'authorProfileImageUrl': "",
        #     'authorChannelUrl':"",
        #     'authorChannelId':{'value':""},
        #     'canRate':False,
        #     'viewerRating':"",
        #     'likeCount':0,
        #     'publishedAt':"",
        #     'updatedAt':"",
        #     'commentId':""
        # })
    
    comment_data = []
    for comment in comments:
        datum = comment['snippet']['topLevelComment']
        comment_id = datum['id']
        comment_info = datum['snippet']
        comment_info['commentId'] = comment_id
        comment_data.append(comment_info)
    
    return pd.DataFrame(comment_data)

In [7]:
# Pass in a list of videos (JSONs) and return a concatenated dataframe. Pass in 1000 video (100000 comments)
def loadVideos(data):
    dataframes = []
    for video in data:
        dataframes.append(getCommentsFromVideo(video))
    
    return pd.concat(dataframes, ignore_index=True)

In [8]:
commentDf = pd.DataFrame()

In [12]:
dataFull = loadVideos(comment_data)

In [14]:
commentsGrouped = pd.DataFrame(dataFull[['videoId', 'textOriginal']].groupby('videoId')['textOriginal'].apply(list)).reset_index()

In [15]:
# commentsGrouped['_id'] = mongoId
# commentsGrouped = commentsGrouped.rename(columns={"_id":"_id", "textOriginal":"comments", "videoId":"id"}).reindex(columns = ['_id', 'comments', 'id'])
commentsGrouped = commentsGrouped.rename(columns={"textOriginal":"comments", "videoId":"id"}).reindex(columns = ['comments', 'id'])

In [16]:
commentsGrouped.iloc[2]

comments    [Helen mom just FYI Andrew the proud father of...
id                                                --wsPKKj_H0
Name: 2, dtype: object

In [17]:
len(commentsGrouped)

5668

In [21]:
commentsJson = commentsGrouped.to_json(orient = 'records')
with open("../pseudoscience-paper-data/groundtruth_videos_comments.json", "w") as f:
    f.write(commentsJson)

In [20]:
5668/6550

0.8653435114503817